<a href="https://colab.research.google.com/github/deardanny/deardanny.github.io/blob/main/n122_chi_square_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://i.imgur.com/RDAD11M.png' width = '200' align = 'right'>

## *DATA SCIENCE / SECTION 1 / SPRINT 2 / NOTE 2*

# 📝 Assignment

# 카이제곱검정


## 1. 공공데이터 조작 연습. 

국가 통계 포털에서 아래 이미지를 참조하여

<https://kosis.kr/statisticsList/statisticsListIndex.do?menuId=M_01_01&vwcd=MT_ZTITLE&parmTabId=M_01_01#SelectStatsBoxDiv>

**2020년 8월**에 해당하는 규모별 미분양현황 에 대한 데이터셋을 생성하세요. 

<img src='https://i.imgur.com/aZc4UJO.png' width = 500>

- `60m이하`, `60~85m`, `85m초과`의 3개 규모와 (column)
- `서울`, `대전`, `대구`, `부산` 4개의 지역을 포함해야합니다. (row)
- 데이터가 없는 경우는 0으로 처리하세요.

이후 데이터셋을 colab으로 불러오세요. 이때 변수의 이름은 `df`를 사용합니다.


In [69]:
col = ['~60m2', '60~85m2', '85m2~']
row = ['서울', '대전', '대구', '부산']

In [70]:
import pandas as pd
import numpy as np
import io


In [7]:
from google.colab import files
myfile = files.upload()

Saving 규모별_미분양현황.csv to 규모별_미분양현황.csv


In [3]:
# 파일 확인 코드
# !ls -al
# 파일 삭제 코드
# !rm 규모별_미분양현황.numbers

In [71]:
def toInt(string):
    return int(string.replace(',',''))

In [72]:
df = pd.read_csv(io.BytesIO(myfile['규모별_미분양현황.csv']), encoding='utf8')
df

# df 정리하기
df = df.iloc[1:, 1:]
df.columns = ['', '-60(공공)', '60-85(공공)', '85-(공공)', '-60(민간)', '60-85(민간)', '85-(민간)']

# 수치형 데이터로 바꾸기
df['-60(공공)'] = df['-60(공공)'].apply(toInt)
df['60-85(공공)'] = df['60-85(공공)'].apply(toInt)
df['85-(공공)'] = df['85-(공공)'].apply(toInt)
df['-60(민간)'] = df['-60(민간)'].apply(toInt)
df['60-85(민간)'] = df['60-85(민간)'].apply(toInt)
df['85-(민간)'] = df['85-(민간)'].apply(toInt)
df


,,-60(공공),60-85(공공),85-(공공),-60(민간),60-85(민간),85-(민간)
1,서울,0,0,0,54,2,0
2,부산,0,0,0,577,735,142
3,대구,0,0,0,143,1437,44
4,대전,0,0,0,782,1,0


## 2. 지역에 대해서 one-sample chi-square test를 실행, 해당 결과를 `chi1`에 저장 후 설명해보세요.

예시) 만약 **9월달 데이터**를 기준으로 한다면
```python
 [0+0+0+52+2+0, 0+0+0+590+665+142, 0+0+0+113+1061+42, 0+0+0+772+1+0]
```
을 비교 하게 될 것입니다.




In [73]:
# 행 별 추출
seoul = df.iloc[0, 1:]
pusan = df.iloc[1, 1:]
daegu = df.iloc[2, 1:]
daejeon = df.iloc[3, 1:]

# 각 데이터를 citysample 변수에 저장하기
citysample = [seoul.sum(), pusan.sum(), daegu.sum(), daejeon.sum()]
citysample

[56, 1454, 1624, 783]

In [74]:
# 참고용
# # Kruskal-Wallis Test (비모수적 평균 비교법) 
# from scipy.stats import kruskal

# kruskal(seoul, pusan, daegu, daejeon)
# # p-value: 0.7098

In [75]:
# one sample chi square test 실행

from scipy import stats
from scipy.stats import normaltest
from scipy.stats import chisquare

chi1 = chisquare(citysample, axis=None)
chi1
# p-value = 0
# 귀무가설: 지역에 대해 미분양 개수가 지역 별로 차이가 없다.
# 대립가설: 미분양 개수가 지역별로 차이가 있다.
# p-value가 0.05보다 작으면 귀무가설을 지지하는 정도야 약해진다.
# 결론: 미분양 개수는 지역별로 차이가 있다.

Power_divergenceResult(statistic=1564.4572376818994, pvalue=0.0)

## 3. 지역과 규모에 대해서 two-sample chi-square test를 실행, 해당 결과를 `chi2`에 저장 후 설명해보세요.

예시) **9월달 데이터**를 기준으로 한다면

| |-60 | 60-85 | 85- |
|:-:|:-:|:-:|:-:|
|서울|52|2|0|
|대전|772|1|0|
|대구|113|1061|42|
|부산|590|665|142|

에 대해서 검정해야 할 겁니다.




In [76]:
# df 다시 정리
df2 = df.copy()

# 공공부문 삭제
df2 = df2.drop(['-60(공공)',	'60-85(공공)',	'85-(공공)'], 1) #axis는 1일때 세로축 삭제 0일때 가로축 삭제

#새로운 컬럼명 부여
df2.columns = ['', '-60', '60-85', '85-']

df2

,,-60,60-85,85-
1,서울,54,2,0
2,부산,577,735,142
3,대구,143,1437,44
4,대전,782,1,0


In [77]:
#카테고리 형으로 바꿔주기
citysample_pri = df2[['-60', '60-85', '85-']].astype('category')
citysample_pri.dtypes

-60      category
60-85    category
85-      category
dtype: object

In [78]:
from scipy.stats import chi2_contingency

print(chi2_contingency(citysample_pri)) # correction 파라미터가 True로 설정: Yates' correction 시행함 

print(chi2_contingency(citysample_pri, correction = False))

citysample2 = chi2_contingency(citysample_pri)
citysample2

(2064.576731417199, 0.0, 6, array([[ 22.24559612,  31.09522594,   2.65917794],
       [577.59101353, 807.36533061,  69.04365586],
       [645.12228746, 901.76155221,  77.11616033],
       [311.04110288, 434.77789124,  37.18100587]]))
(2064.576731417199, 0.0, 6, array([[ 22.24559612,  31.09522594,   2.65917794],
       [577.59101353, 807.36533061,  69.04365586],
       [645.12228746, 901.76155221,  77.11616033],
       [311.04110288, 434.77789124,  37.18100587]]))


(2064.576731417199, 0.0, 6, array([[ 22.24559612,  31.09522594,   2.65917794],
        [577.59101353, 807.36533061,  69.04365586],
        [645.12228746, 901.76155221,  77.11616033],
        [311.04110288, 434.77789124,  37.18100587]]))

In [79]:
chi2 = chisquare(citysample2)
chi2

# p-value = 0
# 귀무가설: 지역과 규모에 따른 미분양 수는 통계학적으로 유의미한 관계가 없다.
# 대립가설: 지역과 규모에 따른 미분양 수가 통계학적으로 유의미한 관계가 있다.
# p-value가 0.05보다 작으면 귀무가설을 지지하는 정도야 약해진다.
# 결론: 지역과 규모에 따른 미분양 수는 독립적이지 않다.
#      따라서, 규모와 지역에 따라 미분양 수 결과에 영향을 끼친다.
# 대립가설 채택: 지역과 규모에 따른 미분양 수가 통계학적으로 유의미한 관계가 있다.

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


Power_divergenceResult(statistic=array([[6055.04171303, 6012.78182895, 6150.66206944],
       [4294.17866432, 3952.42734926, 5838.01925703],
       [4175.6329191 , 3858.22606352, 5802.15878119],
       [4939.89208187, 4601.89455374, 5984.05553266]]), pvalue=array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]]))

### 4. 2번에 대해서 NumPy 를 사용하여 (Scipy를 사용하지 않고) $\chi^2$ test 시행 후 2번의 결과와 비교해보세요. 

- `obs`, `exp`, `chi`라는 변수를 사용해야합니다.

이 후 `chi2_contingency()`의 결과물과 비교해보세요

In [ ]:
# 참고용

# df_t = df.copy()
# df_t = df_t.drop([''], 1)
# df_t = df_t[['-60(공공)','60-85(공공)','85-(공공)', '-60(민간)', '60-85(민간)', '85-(민간)']].astype('category')
# df_t.dtypes

# obs 뽑아보기
# obs = np.array([[0, 0, 0, 54, 2, 0],
#                 [0, 0, 0, 577, 735, 142], 
#                 [0, 0, 0, 143, 1437, 44],
#                 [0, 0, 0, 782, 1, 0]
#                 ])
# total_obs = np.sum(obs)
# total_obs

In [80]:
# 직접 구하기
obs = citysample

exp = np.sum(obs) / 4 # [979.25, 979.25, 979.25, 979.25]

chi = np.sum(np.power(obs - exp, 2) / exp) # chisquare statistics = 1564.4572

squared = np.power(obs - exp, 2)
print(chi)

print(1 - stats.chi2.cdf(chi, df = 4 -1)) #pvalue : 0.0
# 결과값 일치

1564.4572376818994
0.0


## 🔥 도전 과제

아래 세가지를 다하면 됩니다.

### 1. Function

4번에서 사용한 one sample chisquare test를 함수의 형태로 변경하세요.

In [81]:
v1 = [18,22,20,15,23,22]
v2 = [5,23,26,19,24,23]

def myChisq(value):
    
    obs = np.array(value)
    exp = np.sum(obs)/6
    chi = np.sum(np.power(obs - exp, 2) / exp)
    sqaured = np.power(obs - exp, 2)
    pvalue = (1 - stats.chi2.cdf(chi, df = 6-1))
    return chi, pvalue

print(myChisq(v1)) 

# chisquare값과 pvalue를 출력해야합니다.

print(myChisq(v2))

(2.3000000000000003, 0.8062668698851285)
(14.8, 0.011251979028327308)


### 2. ANOVA

아래 링크를 참조하여 ANOVA 에 대한 글을 읽고

<https://partrita.github.io/posts/ANOVA-python/>

다음 `4개 그룹에 대해서 평균의 차이가 있는지`에 대한 가설 검정을 시행하세요.

A : `38 33 35 92 76 97 88 41 11  9`

B : `18 52 62 48 30 40 87 12 97 82`

C :  `28 90  5 49 66 73 96 80  4 17`

D : ` 8 99  4 12  7 64 18 10  9 20`



In [82]:
import scipy.stats as stats
import pandas as pd
import urllib
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# DF anova
anova = {
        'A':[38,33,35,92,76,97,88,41,11,9],
        'B':[18,52,62,48,30,40,87,12,97,82],
        'C':[28,90,5,49,66,73,96,80,4,17],
        'D':[8,99,4,12,7,64,18,10,9,20]
         }

# group 나누기
group1 = anova['A']
group2 = anova['B']
group3 = anova['C']
group4 = anova['D']

# ANOVA Test
F_statistic, pVal = stats.f_oneway(group1, group2, group3, group4)

print(F_statistic, pVal)

# p-value = 0.1792
# 귀무가설: 4개 그룹에 대해서 평균의 차이가 없다.
# 대립가설: 4개 그룹에 대해서 평균의 차이가 있다.
# p-value가 0.05보다 작으면 귀무가설을 지지하는 정도야 약해진다.
# 결론: 4개 그룹에 대해서 평균의 차이가 없다.

1.7249594239128412 0.17920877113948797


## 3. Sign Test

아래 링크를 참조하여 Sign Test에 대한 글을 읽고

<https://en.wikipedia.org/wiki/Sign_test>

다음 데이터에서 두 그룹간에 차이가 있다에 대한 Sign Test의 과정과 결과를 설명하세요

|A|B|
|:-:|:-:|
|142|138|
|140|136|
|144|147|	
|144|139|	
|142|143|	
|146|141|	
|149|143|	
|150|145|	
|142|136|	
|148|146|	



In [83]:
from scipy.stats import wilcoxon

# DF sign
sign = {
        'A':[142,140,144,144,142,146,149,150,142,148],
        'B':[138,136,147,139,143,141,143,145,136,146]
       }
    
#DF 나누기
sign1 = sign['A']
sign2 = sign['B']

sign_w, sign_p = wilcoxon(sign1, sign2)

print(w, p)

# p-value = 0.01618
# 귀무가설:  두 그룹간에 차이가 없다.
# 대립가설:  두 그룹간에 차이가 있다.
# p-value가 0.05보다 작으면 귀무가설을 지지하는 정도야 약해진다.
# 결론: 두 그룹간에 차이가 없다.

4.0 0.016184453851159696


In [17]:
obs

[56, 1454, 1624, 783]

In [18]:
exp

979.25

In [20]:
chi

1564.4572376818994